In [1]:
## Imports

import pandas as pd
import io
import matplotlib.pyplot as plt
import time
from sklearn.metrics import roc_auc_score

import re
import string
import nltk
import itertools
import numpy as np

# this allows you to apply a progress bar to pandas operations
from tqdm import tqdm
tqdm.pandas()

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Github repo installation of paddle
!python -m pip install paddlepaddle -i https://mirror.baidu.com/pypi/simple
# Install paddle OCR
!pip install paddleocr
# git clone
!git clone https://github.com/PaddlePaddle/PaddleOCR.git
# cv2
!pip install "opencv-python-headless<4.3"


Looking in indexes: https://mirror.baidu.com/pypi/simple
fatal: destination path 'PaddleOCR' already exists and is not an empty directory.


In [3]:
import cv2
from paddleocr import PaddleOCR, draw_ocr
from matplotlib import pyplot as plt
import os

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/paddle/vision/transforms/functional_pil.py:36: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': Image.NEAREST,
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/paddle/vision/transforms/functional_pil.py:37: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  'bilinear': Image.BILINEAR,
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/paddle/vision/transforms/functional_pil.py:38: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  'bicubic': Image.BICUBIC,
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/paddle/vision/transforms/functional_pil.py:39: DeprecationWarning: BOX is deprecated and will be removed in Pillo

In [4]:
ocr_model = PaddleOCR(lang='en')

def extract_text(image_path):
    result = ocr_model.ocr(image_path)
    
    return [res[1][0] for res in result]

[2022/09/06 14:10:50] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, shape_info_filename=None, precision='fp32', gpu_mem=500, image_dir=None, det_algorithm='DB', det_model_dir='/home/studio-lab-user/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_box_type='quad', det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, det_fce_box_type='poly', rec_algorithm='SVTR_LCNet', rec_model_dir='/home/studio-lab-user/.paddleocr/whl/rec/en/en_PP-OCRv3_rec_infer', rec_image_shape='3, 48, 320', rec_bat

---    Fused 0 subgraphs into layer_norm op.
---    Fused 0 subgraphs into layer_norm op.


In [5]:
# To pull in previusly save data
import pandas as pd
data = pd.read_csv("image_relevant.csv", encoding='latin')
data.head()

,Unnamed: 0,id,description,image,images,relevant,processed_description,text_score
0,0,918707,Boys Among Us Hoodies Super Cool Print Clothes...,968bf5e6c99211c2ad8ac27fa55e4651b941d137,['968bf5e6c99211c2ad8ac27fa55e4651b941d137'],1,boys among us hoodies super cool print clothes...,0.192416
1,1,918708,Cat And Dog Cardboard Printing Jigsaw Puzzle M...,68ece3fab138feb537eee296f3348f3543e23a1d,"['68ece3fab138feb537eee296f3348f3543e23a1d', '...",0,cat dog cardboard printing jigsaw puzzle manuf...,0.030835
2,2,918709,0098 30 Pcs 21pack Sensory 4 Piece Puzzle Fidg...,a31d8c30452d17221b0d8d4f9ccf50bf8dfff0cf,"['a31d8c30452d17221b0d8d4f9ccf50bf8dfff0cf', '...",0,0098 30 pcs 21pack sensory 4 piece puzzle fidg...,0.001173
3,3,918710,Boys Sweatshirt Among us 3D Hoodies Cool Fashi...,a278812a832f24a822e2602b91a7b1f5749926cb,['a278812a832f24a822e2602b91a7b1f5749926cb'],1,boys sweatshirt among us 3d hoodies cool fashi...,0.275841
4,4,918711,20cm Among Us Soft Plush Toys Vocal Butt Toy S...,3f7fc0f877323f4b74cb9a4c3847d3eea6f3ae5a,['3f7fc0f877323f4b74cb9a4c3847d3eea6f3ae5a'],0,20cm among us soft plush toys vocal butt toy s...,0.000861


In [6]:
%cd /home/studio-lab-user/DS-Projects/OCR

/home/studio-lab-user/DS-Projects/OCR


In [7]:
image_df = data[['image','relevant']]

In [8]:
image_df = image_df
image_df

,image,relevant
0,968bf5e6c99211c2ad8ac27fa55e4651b941d137,1
1,68ece3fab138feb537eee296f3348f3543e23a1d,0
2,a31d8c30452d17221b0d8d4f9ccf50bf8dfff0cf,0
3,a278812a832f24a822e2602b91a7b1f5749926cb,1
4,3f7fc0f877323f4b74cb9a4c3847d3eea6f3ae5a,0
...,...,...
464,43a18980d983e7835ce8a9f50ae8953390197f73,0
465,a2db2c697881750d70feaa1a90d4f271338b9337,0
466,077d6694396e202064fb5b01b42837d2949e4ce7,0
467,a8057438d83d4fc141347237c23e2d3fc714d43f,0


In [9]:
# # OCR_images에 있는 리스트들로 컬럼 하나 만들기. 리스트 이름 따기.
# import pandas as pd
# import os
# image_df = pd.DataFrame(os.listdir(), columns=['image'])

In [10]:
# image_dff
# image_dff.drop(index=0, inplace=True)

In [11]:
# image_list = image_dff['image'].tolist()
# len(image_list)

In [12]:
# text_list = []
# for image_path in image_list:
#     result = ocr_model.ocr(image_path)
#     text = [res[1][0] for res in result]
#     text_list.append(text)
# print(text_list)

In [8]:
image_path = f"/home/studio-lab-user/DS-Projects/OCR/SWOOP_OCR_images/000e7696e9936c1f2d2e2214cdfbdb54b8878374.jpg"


In [9]:
image_path = f"/home/studio-lab-user/DS-Projects/OCR/SWOOP_OCR_images/000e7696e9936c1f2d2e2214cdfbdb54b8878374.jpg"
result = ocr_model.ocr(image_path)
texts = [res[1][0] for res in result]
texts

[2022/09/05 13:32:10] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process
[2022/09/05 13:32:11] ppocr DEBUG: dt_boxes num : 3, elapse : 1.0296854972839355
[2022/09/05 13:32:12] ppocr DEBUG: rec_res num  : 3, elapse : 0.267653226852417


['VWoud n2', 'AMONG US', 'IMPOSTOR']

In [10]:
result

[[[[125.0, 353.0], [181.0, 360.0], [148.0, 608.0], [92.0, 601.0]],
  ('VWoud n2', 0.818354606628418)],
 [[[617.0, 361.0], [672.0, 354.0], [706.0, 601.0], [651.0, 608.0]],
  ('AMONG US', 0.9929673671722412)],
 [[[275.0, 458.0], [512.0, 458.0], [512.0, 492.0], [275.0, 492.0]],
  ('IMPOSTOR', 0.9865270853042603)]]

In [408]:
# final = []
# for word in texts:
#     final.extend(word.split())
# final

['VWoud', 'n2', 'AMONG', 'US', 'IMPOSTOR']

In [13]:
def get_words_and_scores_and_add_to_dataframe(row):
    photo = row["image"]
    if photo == "empty":
        return [[],[]]
    final = []
    image_path = f"/home/studio-lab-user/DS-Projects/OCR/SWOOP_OCR_images/{photo}.jpg"
    result = ocr_model.ocr(image_path)
    texts = [res[1][0] for res in result]
    # for word in texts:
    #     final.extend(word.split())
    scores = [res[1][1] *100 for res in result]
    result = [texts, scores]
    return result

In [14]:
# image_dff_100 = image_dff[100:150]

In [15]:
image_df["words"] = image_df.apply(get_words_and_scores_and_add_to_dataframe, axis=1)
image_df.head()

[2022/09/06 14:10:50] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process
[2022/09/06 14:10:51] ppocr DEBUG: dt_boxes num : 2, elapse : 0.7083141803741455
[2022/09/06 14:10:51] ppocr DEBUG: rec_res num  : 2, elapse : 0.1545708179473877
[2022/09/06 14:10:51] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process
[2022/09/06 14:10:52] ppocr DEBUG: dt_boxes num : 1, elapse : 0.8279731273651123
[2022/09/06 14:10:52] ppocr DEBUG: rec_res num  : 1, elapse : 0.10609889030456543
[2022/09/06 14:10:52] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process
[2022/09/06 14:10:52] ppocr DEBUG: dt_boxes num : 0, elapse : 0.07217240333557129
[2022/09/06 14:10:52] ppocr DEBUG: rec_res num  : 0, elapse : 1.1920928955078125e-06
[2022/09/06 14:10:52] ppocr WARNING: Since the angle classifi

KeyboardInterrupt: 

In [ ]:
import re
import string

def clean_text(row):
    word_list = row["words"][0]
    new_word_list = []
    for word in word_list:
        new_word = word.lower()
        new_word = re.sub("[%s]" % re.escape(string.punctuation), " ", new_word)
        new_word_list.append(new_word)
    return [new_word_list, row["words"][1]]

In [ ]:
image_df["words"] = image_df.apply(clean_text, axis=1)

In [ ]:
image_df

In [ ]:
## define the target words / phrases

targets = ["among us", "impostor"]

In [ ]:
# helper functions for funal scoring function

def advance(iterator, step):
    next(itertools.islice(iterator, step, step), None)

def tuplewize(iterable, size):
    iterators = itertools.tee(iterable, size)
    for position, iterator in enumerate(iterators):
        advance(iterator, position)
    return zip(*iterators)

In [ ]:
# define function to produce final scores

def get_final_score(row):
    sample_list = row["words"][0]
    score_list = row["words"][1]
    max_score = 0
    
    for target in targets:
        phrase_length = len(target.split())
        for c, s in zip(tuplewize(sample_list, phrase_length), tuplewize(score_list, phrase_length)):
            new_sample = " ".join(c)
            print(new_sample)
            edit_distance = nltk.edit_distance(target, new_sample)
            closeness = (len(target)-edit_distance)/len(target)
            confidence_score = np.mean(s)
            final_score = closeness * confidence_score
            if final_score > max_score:
                max_score = final_score
             
    return max_score/100

In [ ]:
import itertools
import nltk
import numpy as np

image_df["OCR_score"] = image_df.apply(get_final_score, axis=1)

In [ ]:
image_df = image_df.sort_values(by='OCR_score', ascending=False)
pd.set_option('display.max_rows', None)
len(image_df)

In [ ]:
RA_score = roc_auc_score(image_df["relevant"], image_df["OCR_score"])

print("Rekognition ROC AUC Score =  %.2f" % (RA_score))

In [ ]:
image_df

In [83]:
image_df.to_csv('/home/studio-lab-user/DS-Projects/OCR/paddle_ocr_v2.csv')

In [84]:
# Reference https://www.youtube.com/watch?v=t5xwQguk9XU

In [85]:
# Dataframe of true positives

tp = image_df.loc[(image_df['relevant'] == 1) & (image_df['OCR_score'] >= 0.5)] 
print(len(tp))

119


In [86]:
# Dataframe of true negatives

tn = image_df.loc[(image_dff['relevant'] == 0) & (image_df['OCR_score'] < 0.5)] 
print(len(tn))

317


In [87]:
# Dataframe of false positives

fp = image_df.loc[(image_df['relevant'] == 0) & (image_df['OCR_score'] >= 0.5)] 
print(len(fp))

1


In [88]:
# Dataframe of false negatives

fn = image_df.loc[(image_df['relevant'] == 1) & (image_df['OCR_score'] < 0.5)] 
print(len(fn))

32


In [89]:
# Precision (What % of irdentified positives and true?)
precision = len(tp) / (len(tp) + len(fp))
print(precision)

0.9916666666666667


In [90]:
# PRecall (% of true positives identified)
recall = len(tp) / (len(tp) + len(fn))
print(recall)

0.7880794701986755


In [91]:
fn

,image,relevant,words,OCR_score
163,04437a357f2bbc9606cda30825366bb76b0559f2,1,"[[l m, your, fabher, 00000, rimong us, xvamafn...",0.499820
285,1e2688724847e6ace895252f804fc30258f92b12,1,"[[mong, whoisit, nol711], [56.05505704879761, ...",0.448440
224,9312a0394cdca0f19e08f589fb7d5d10b7b73660,1,"[[g, us, s, xvamafil, i m, your, fabher], [92....",0.410818
101,f5dd445f3fff531523efd5fa98a5b1d7700a4aed,1,"[[trick, 2, i d, skip, vote, that, kini, rgenc...",0.399637
375,587864639734fb3fbc29cfd53ff4dfdfba662acd,1,"[[smang], [61.39713525772095]]",0.368383
29,d5e94eee21b701f5f3d409bb1b9bc43a4b47096c,1,"[[smang], [61.39713525772095]]",0.368383
86,dc35ae67c9ed08081e7fe8592924ab3804840626,1,"[[wond, n2, mong, us], [81.32509589195251, 97....",0.335962
192,1f2d04f5078f7b2b6bc4e521b75e9537723c518b,1,"[[vwona, n2, among, us], [80.08106350898743, 9...",0.333739
223,9538489727bd15088eb41df30792a695eba59ad2,1,"[[wond, n2, mong, us], [78.60596776008606, 97....",0.330741
77,03613ec8794b6bf98f4ae263cafdccf6d054ee0d,1,"[[vwon , n2, among, us], [80.00211119651794, 9...",0.329893


In [92]:
print(image_df.loc[86]['words'])
print(image_df.loc[86]['OCR_score'])

[['wond', 'n2', 'mong', 'us'], [81.32509589195251, 97.85457849502563]]
0.33596188947558403
